In [2]:
from tst.domutil.util import *
from Bio import SearchIO
INPUT = full('$SEQlib/hmm/CATHS100.domtbl')
INPUT = full('$SEQlib/hmm/CATHS100_v410.domtbl')
f_handles  = split_file(INPUT,number = 6 )  
parser = SearchIO.parse( INPUT, 'hmmsearch3-domtab' )

seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"

c0 = counter([],INF=1,per = 2)

            
acc2hmm_raw  = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))
acc2seq_raw = dict(seqDB_curr.sequence_set.filter().values_list('acc','id'))



/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [1]:
from tempfile import TemporaryFile , mkdtemp

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def split_file(fname,length = None, number = None):
    if length and number:
        raise Exception('You can only specify one of length or number')
    f_handles = []
#     if length:
    tempdir = mkdtemp(prefix = '/tmp/feng')
    
    if number:
        lcount = file_len(fname)
        nlcount = lcount // number + 1
        with open(fname) as f:
            lines = f.readlines()
        idxs = range(0, lcount, nlcount)
        for i,idx in enumerate(idxs):
#             f = TemporaryFile()
            fname = tempdir + '/%d'%i
            f = open(fname, 'w+')
            f.write( ''.join(lines[idx: idx+nlcount]) )
            f.seek(0)
            f_handles.append(fname)
        
    return f_handles
if __name__ == '__test__':
    lc1 = len(open(INPUT,'r').read())
    lc2 = len(''.join(f.read() for f in f_handles))
    assert  lc2 == lc1
            

import sys

def parse_worker(fakefile):
#     print >> sys.__stdout__,fakefile.closed
    parser = SearchIO.parse(fakefile,'hmmsearch3-domtab')
    for q_hits in parser:
    #     print q_hits
        acc = p_cathdomain.findall(q_hits.id)[0]

        try:
            c1 = counter([],INF=-1,ifprint = 0 )
            query_id = acc2hmm[ acc ]
            Jdict = {'query_id':query_id,
                    'hits' : [], }
            for hit in q_hits:
                hsp = hit[0]
                target_acc = p_cathdomain.findall(hit.id)[0]
                try:
                    jdict = hsp2jdict( hsp,simple = 1)
                    jdict['query_id'] = query_id
                    jdict['target_id'] = acc2seq[target_acc]
                    Jdict['hits'].append(jdict)
                except BaseException as e:
                    c1.fail('', target_acc)
                finally:
                    c1.count()

            if c1.f:
                c0.fail('%d of %d instances failed for HMM %s \n Most Recent Exception' % (c1.f, c1.f, acc, c1.e ), acc)
            q.put( Jdict )


        except BaseException as e:
            try:
                c0.fail('%s failed for %s'%(q_hits.id, e), q_hits.id)
            except:
                pass
        finally:
            c0.count()

            
            
def db_listener():
    c = counter([],INF=1, stdout = sys.__stdout__)
    with transaction.atomic():
        while 1:
            
            try:
                obj = q.get()
                c.count()
                if not obj:
                    break
                else:
                    hmm = HMMprofile.objects.defer('text').get(id = obj['query_id'])
                    oldhits = list(
                        hmm.hits.values_list('id',flat = True)
                    )
                    bulk = []

                    for jdict in obj['hits']:
                        if jdict['target_id'] in oldhits:
                            continue
                        else:
                            db_hit = hit4hmm2hsp(**jdict)
                            bulk.append( db_hit )
#                             db_hit.save()
                    hit4hmm2hsp.objects.bulk_create( bulk )
                    
            except BaseException as e:
                print e
                c.fail('%s'%e,)
#                 break
    return c
# tpf=TemporaryFile('w+')


In [24]:
# f_handles = [split_file(INPUT,number = 30)[0]]
# f0 = f_handles[0]


In [3]:
# f_handles = [f0]
ts = []
# for pcount in [9,10,11,12]:
pcount = 9
import sys
if 1:
    f_handles = split_file( INPUT,number = pcount - 2)
#     f_handles = split_file( f0,number = pcount - 2)


    import os
    import multiprocessing as mp
    # from 
    from multiprocessing.managers import BaseManager, SyncManager
    # class MyManager(BaseManager): pass
    class MyManager(SyncManager): pass
    MyManager.register('counter',counter)
    from time import time

    t0 = time()
    if __name__=='__main__':
    #     m = mp.Manager()
        m = MyManager()
        m.start()
    #     raise Exception
    #     manager.start()
        c0 = m.counter([],INF=1, ifprint = 1)
        acc2hmm = m.dict(acc2hmm_raw)
        acc2seq = m.dict(acc2seq_raw)

        q = m.Queue();          

        if 1:
            pool = mp.Pool( pcount )
    #     with mp.Pool(mp.cpu_count() - 1) as pool:
            watcher  = pool.apply_async(db_listener, args = [])
            jobs = []
            for f_handle in f_handles:
                print f_handle
    #         for f_handle in [INPUT]:

                job  = pool.apply_async(parse_worker, args = (f_handle,))
    #             print f_handle.closed
                jobs.append(job)
    #         for f_handle in 

            for job in jobs:
                job.get()
            q.put(None)
            watcher.get()

        for f in f_handles:
            os.remove(f)




    #     pool.close()
    #     pool.join()
    #     watcher = pool.apply_async( db_listener, (q, c));
    #     jobs = [];
    #     def f(it):
    #         return (it)
    #     parser = pool.imap(, parser)
    t = time() - t0
    ts.append(t)
    print pcount, t 
    
    # print time() - t0

0 of -1
 0 of -1
/tmp/fengCDu_nd/0
/tmp/fengCDu_nd/1
/tmp/fengCDu_nd/2
/tmp/fengCDu_nd/3
/tmp/fengCDu_nd/4
/tmp/fengCDu_nd/5
/tmp/fengCDu_nd/6
cath|4_1_0|3ozqA01/14-50_188-292_364-389 failed for '3ozqA01'
cath|4_1_0|3ozqA02/51-187_293-331 failed for '3ozqA02'
100 of -1
 100 of -1
cath|4_1_0|1a7cA02/171-280_326-349 failed for '1a7cA02'
200 of -1
cath|4_1_0|1qrvA00/2-74 failed for '1qrvA00'
300 of -1
400 of -1
500 of -1
600 of -1
700 of -1
800 of -1
 800 of -1


tst/domutil/util.py:339: RuntimeWarning: divide by zero encountered in log10
  jdict["logCevalue"] = max(-1000,np.log10(jdict.pop("evalue_cond")))
tst/domutil/util.py:340: RuntimeWarning: divide by zero encountered in log10
  jdict["logIevalue"] = max(-1000,np.log10(jdict.pop("evalue")))


900 of -1
1000 of -1
cath|4_1_0|3fghA00/111-177 failed for '3fghA00'
cath|4_1_0|3fgqA01/22-49_189-294_345-397 failed for '3fgqA01'
cath|4_1_0|3fgqA02/50-188_295-344 failed for '3fgqA02'
1100 of -1
1200 of -1
1300 of -1
 1300 of -1
1400 of -1
1500 of -1
1600 of -1
1700 of -1
1800 of -1
1900 of -1
 1900 of -1
 1900 of -1
2000 of -1
 2000 of -1
 2000 of -1
cath|4_1_0|1sekA02/180-286_333-342 failed for '1sekA02'
2100 of -1
2200 of -1
2300 of -1
 2300 of -1
cath|4_1_0|3pzfA02/57-199_300-353 failed for '3pzfA02'
2400 of -1
cath|4_1_0|1sngA01/8-35_168-258_333-361 failed for '1sngA01'
2500 of -1
2600 of -1
2700 of -1
2800 of -1
2900 of -1
3000 of -1
3100 of -1
3200 of -1
3300 of -1
3400 of -1
3500 of -1
3600 of -1
 3600 of -1
3700 of -1
3800 of -1
3900 of -1
 3900 of -1
4000 of -1
cath|4_1_0|2wxyC01/73-101_257-350_406-450 failed for '2wxyC01'
4100 of -1
cath|4_1_0|2wxyC02/102-256_351-405 failed for '2wxyC02'
4200 of -1


tst/domutil/util.py:339: RuntimeWarning: divide by zero encountered in log10
  jdict["logCevalue"] = max(-1000,np.log10(jdict.pop("evalue_cond")))
tst/domutil/util.py:340: RuntimeWarning: divide by zero encountered in log10
  jdict["logIevalue"] = max(-1000,np.log10(jdict.pop("evalue")))


4300 of -1
4400 of -1
4500 of -1
4600 of -1
4700 of -1
4800 of -1
4900 of -1
cath|4_1_0|1f0cA01/51-189_294-337_348-358 failed for '1f0cA01'
5000 of -1
cath|4_1_0|2h09A02/95-150 failed for '2h09A02'
5100 of -1
cath|4_1_0|1f5tA02/1076-1121 failed for '1f5tA02'
5200 of -1
5300 of -1
cath|4_1_0|2h4pA02/201-302 failed for '2h4pA02'
cath|4_1_0|2xn6A01/19-190_287-356 failed for '2xn6A01'
cath|4_1_0|2xn6A02/191-286 failed for '2xn6A02'
5400 of -1
5500 of -1
 5600 of -1
5600 of -1
 5600 of -1
5700 of -1
5800 of -1
cath|4_1_0|1v63A00/1-101 failed for '1v63A00'
5900 of -1
6000 of -1
6100 of -1
6200 of -1
 6200 of -1
6300 of -1
 6300 of -1
6400 of -1
6500 of -1
6600 of -1
6700 of -1
6800 of -1
6900 of -1
 6900 of -1
7000 of -1
7100 of -1
7200 of -1
7300 of -1
 7300 of -1
7400 of -1
7500 of -1
7600 of -1
7700 of -1
cath|4_1_0|3stoA01/16-196_297-350 failed for '3stoA01'
cath|4_1_0|3stoA02/197-296_351-403 failed for '3stoA02'
7800 of -1
7900 of -1
8000 of -1
cath|4_1_0|1wgfA01/26-78 failed for '1wgfA

In [5]:
### Manually delete temp files if crashed
for f in f_handles:
    os.remove(f)

OSError: [Errno 2] No such file or directory: '/tmp/fengCDu_nd/0'

In [4]:
print hit4hmm2hsp.objects.count()

5235545


In [ ]:
print hit4hmm2hsp.objects.count()
hit4hmm2hsp.objects.all().delete()
print hit4hmm2hsp.objects.count()

5233048


In [5]:
hi = '''cath|4_1_0|4jx4C02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4jx4D02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4jx5A02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4jx5B02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4jx5C02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4jx5D02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4locA02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4locB02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4locC02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4locD02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4m6vA02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4m6vB02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4m6vC02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4m6vD02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfdA02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfdB02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfdC02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfdD02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfeA02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfeB02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfeC02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mfeD02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mimA02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mimB02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mimC02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|4mimD02/537-824         -            288 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.8   10.3   3.1   1   1     0.061       8.2    8.8   3.1    16   129    95   247    73   250 0.79 -
cath|4_1_0|1q7cA00/2-244           -            244 cath|4_1_0|8abpA01/2-108_255-285 -            137       2.9   10.3   4.0   1   1       0.1        14    8.0   4.0    17    83    14    82     4   231 0.88 -
cath|4_1_0|2qf7A04/517-826         -            310 cath|4_1_0|8abpA01/2-108_255-285 -            137       3.6    9.9   3.1   1   1     0.076        10    8.5   3.1    16   129   115   267    93   270 0.78 -
cath|4_1_0|2qf7B05/517-826         -            310 cath|4_1_0|8abpA01/2-108_255-285 -            137       3.6    9.9   3.1   1   1     0.076        10    8.5   3.1    16   129   115   267    93   270 0.78 -
cath|4_1_0|4mjmA00/4-470           -            336 cath|4_1_0|8abpA01/2-108_255-285 -            137       8.8    8.7   7.1   1   1      0.32        44    6.4   5.1    26   113   136   223   108   259 0.68 -
cath|4_1_0|4lvuA00/18-271          -            254 cath|4_1_0|8abpA01/2-108_255-285 -            137       9.2    8.6   8.5   1   1      0.29        39    6.6   8.5    19    94    23   112    16   182 0.65 -
cath|4_1_0|3kqnA03/431-451_484-625 -            163 cath|4_1_0|8ohmA03/431-451_484-624 -            140   1.1e-72  244.5   1.9   1   1   1.4e-76   1.3e-72  244.3   1.9     1   140    17   161    17   161 0.99 -
cath|4_1_0|2f55A03/431-451_484-624 -            162 cath|4_1_0|8ohmA03/431-451_484-624 -            140   1.3e-72  244.3   1.3   1   1   1.7e-76   1.5e-72  244.1   1.3     1   140    17   161    17   161 0.99 -
cath|4_1_0|2zjoA03/431-451_484-630 -            175 cath|4_1_0|8ohmA03/431-451_484-624 -            140     3e-72  243.1   2.2   1   1   3.9e-76   3.5e-72  242.9   2.2     1   140    17   161    17   161 0.99 -
cath|4_1_0|1a1vA03/432-451_484-624 -            161 cath|4_1_0|8ohmA03/431-451_484-624 -            140   6.7e-72  242.0   1.5   1   1   8.5e-76   7.6e-72  241.8   1.5     1   140    16   160    16   160 0.99 -
cath|4_1_0|1heiA03/432-451_484-620 -            157 cath|4_1_0|8ohmA03/431-451_484-624 -            140   4.9e-70  235.9   0.9   1   1   6.2e-74   5.6e-70  235.8   0.9     1   137    16   157    16   157 0.99 -
cath|4_1_0|8ohmA03/431-451_484-624 -            162 cath|4_1_0|8ohmA03/431-451_484-624 -            140   6.3e-70  235.6   1.3   1   1     8e-74   7.1e-70  235.4   1.3     2   140    18   161    17   161 0.99 -
cath|4_1_0|3o8bA05/487-620         -            134 cath|4_1_0|8ohmA03/431-451_484-624 -            140   1.7e-66  224.5   0.6   1   1   2.1e-70   1.8e-66  224.4   0.6     9   137     1   134     1   134 0.99 -
cath|4_1_0|1mngA01/21-91           -             71 cath|4_1_0|8ohmA03/431-451_484-624 -            140      0.13   14.2   0.0   1   1   1.5e-05      0.13   14.2   0.0    62   102     8    50     2    69 0.82 -
cath|4_1_0|1oy0A00/18-279          -            264 cath|4_1_0|8ohmA03/431-451_484-624 -            140      0.38   12.7   0.0   1   1   8.2e-05      0.73   11.8   0.0    62    90     2    30     2    37 0.90 -
'''


# hi = '''#                                                                                            --- full sequence --- -------------- this domain -------------   hmm coord   ali coord   env coord
# # target name                    accession   tlen query name               accession   qlen   E-value  score  bias   #  of  c-Evalue  i-Evalue  score  bias  from    to  from    to  from    to  acc description of target
# #            ------------------- ---------- -----     -------------------- ---------- ----- --------- ------ ----- --- --- --------- --------- ------ ----- ----- ----- ----- ----- ----- ----- ---- ---------------------
# '''
# with open('tmp','w') as f:
#     f.write(hi)

import io
f = io.StringIO()
f.write( unicode(hi) )

# f.write( hi )

g = SearchIO.parse( f ,'hmmsearch3-domtab' )
print next(g)
print next(g)

StopIteration: 

In [45]:
class X(object):
  def __init__(self, x):
    pass

  def doit(self, bar):
    pass

class Y(X):
  def __init__(self):
    super(Y, self).__init__(123)

  def doit(self, foo):
    return super(Y, self).doit(foo)
# Y()

In [50]:
# del fakefile
# type(StringIO)
StringIO?

In [68]:
with open('tmp','w') as f:
    f.write(hi)
l = open('tmp','r').readlines()

In [2]:
from StringIO import StringIO
class fakefile():
    def __init__(self):
#         super( fakefile , self)
#         SuperClass.__init__()
        self.s = StringIO()
        self.write = self.s.write
        self.read = self.s.getvalue
        self.readlines = lambda : ['%s/n'%x for x in self.s.getvalue().splitlines()]
#     def readlines(self):
# dir(fakefile())
# f = fakefile()
# f.write(hi)
# f.readlines()
# # f.read()
#     io.StringIO

In [34]:
f = io.StringIO()
f.write( unicode(hi) )
print f.read()
# import io


In [12]:
from tst.domutil.util import *
INPUT = full('$SEQlib/hmm/CATHS100.domtbl')
with open(INPUT,'r') as f:
    g = f.readlines()

In [14]:
# type(g)
len(g)

5538468